In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Colab_Pro/DLCV/HW2/stable-diffusion

/content/drive/MyDrive/Colab_Pro/DLCV/HW2/stable-diffusion


In [3]:
!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113
!pip install -r ../requirements.txt
!pip install -e .

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 GB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 96.3 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.5.0+cu121
    Uninstalling torch-2.5.0+cu121:
      Successfully uninstalled torch-2.5.0+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.20.0+cu121
    Uninstalling torchvision-0.20.0+cu121:
      Successfully uninstalled torchvision-0.20.0+cu121
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.5.0+cu121
    Uninstalling torchaudio-2.5.0+cu121:
      Successfully uninstalled torchaudio-2.5.0+cu121
Obtaining taming-transformers from git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers (from -r ../

In [4]:
!pip install pytorch-lightning==1.4.2
!pip install omegaconf==2.0.0
!pip install einops==0.3.0
!pip install torchmetrics==0.6.0
!pip install ftfy
!pip install kornia==0.6

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 916.6/916.6 kB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.6/890.6 kB 57.1 MB/s eta 0:00:00
  Attempting uninstall: einops
    Found existing installation: einops 0.8.0
    Uninstalling einops-0.8.0:
      Successfully uninstalled einops-0.8.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.4/329.4 kB 20.2 MB/s eta 0:00:00
  Attempting uninstall: torchmetrics
    Found existing installation: torchmetrics 1.5.1
    Uninstalling torchmetrics-1.5.1:
      Successfully uninstalled torchmetrics-1.5.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.1/367.1 kB 22.0 MB/s eta 0:00:00


## Import Packages

In [5]:
import os
import argparse
import json
import glob

import torch
from torch import nn, optim
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torchvision import  transforms
import numpy as np
from omegaconf import OmegaConf
from ldm.util import instantiate_from_config
from ldm.models.diffusion.dpm_solver import DPMSolverSampler

Colab: To access 'src/taming-transformers/' &  'src/clip/'

In [6]:
import sys
import os

# Assuming you are running this from the stable-diffusion directory
sys.path.append(os.path.join(os.getcwd(), 'src/taming-transformers/'))
sys.path.append(os.path.join(os.getcwd(), 'src/clip/'))

In [7]:
import pytorch_lightning as pl
print(pl.__version__)
print(torch.__version__)

1.4.2
1.11.0+cu113


## Parameters

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# batch_size & training epochs
BATCH_SIZE = 1
n_epoch = 500
train_concept = "david_revoy"
fine_tune = True # fine-tune trained embedding with high score but not pass

In [9]:
ckpt = "ldm/models/stable-diffusion-v1/model.ckpt"
H = 512
W = 512
jsonfile='../hw2_data/textual_inversion/input.json'
dog_dir="../hw2_data/textual_inversion/0"
david_revoy_dir="../hw2_data/textual_inversion/1"

## utils

In [10]:
# (ref: txt2img.py)
def load_model_from_config(config, ckpt, verbose=False):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cpu")
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)

    model.to(device)
    model.eval()
    return model


def prepare_training_data(token_names, image_dirs):
    """token_names is list of special token, ex: [<new1>, <new2>]
    image_dirs is list of image dir"""
    training_data = {}
    for i, image_dir in enumerate(image_dirs):
        image_files = glob.glob(os.path.join(image_dir, "*.jpg"))
        for j, image_file in enumerate(image_files):
            training_data[f"{i}_{j}"] = {
                "image_path": image_file,
                "prompt": f"A photo of {token_names[i]}"
            }
    return training_data


def encode_image_to_latent(image_tensors):
    """Encode an image using the model. return target latent"""
    #image_tensors.shape = N*C*H*W
    latent = model.encode_first_stage(image_tensors) # Encode to latent space  # first stage encoder: x to z
    return model.get_first_stage_encoding(latent).detach()


class TextualInversionDataset(Dataset):
    def __init__(self, token_names, image_dirs, transform=None):
        self.transform = transform

        training_data = prepare_training_data(token_names, image_dirs)
        """print(training_data)
        {'0_0': {'image_path': '../hw2_data/textual_inversion/0/01.jpg', 'prompt': 'A photo of <new1>'},
        '0_1': {'image_path': '../hw2_data/textual_inversion/0/04.jpg', 'prompt': 'A photo of <new1>'},
        '0_2': {'image_path': '../hw2_data/textual_inversion/0/02.jpg', 'prompt': 'A photo of <new1>'},
        '0_3': {'image_path': '../hw2_data/textual_inversion/0/00.jpg', 'prompt': 'A photo of <new1>'},
        '0_4': {'image_path': '../hw2_data/textual_inversion/0/03.jpg', 'prompt': 'A photo of <new1>'},

        '1_0': {'image_path': '../hw2_data/textual_inversion/1/plxthumbnailer-1.jpg', 'prompt': 'A photo of <new2>'},
        '1_1': {'image_path': '../hw2_data/textual_inversion/1/2018-07-01_Adventure_by-David-Revoy.jpg', 'prompt': 'A photo of <new2>'},
        '1_2': {'image_path': '../hw2_data/textual_inversion/1/plxthumbnailer.jpg', 'prompt': 'A photo of <new2>'},
        '1_3': {'image_path': '../hw2_data/textual_inversion/1/2022-05-31_ep37-panel1-rendered_net.jpg', 'prompt': 'A photo of <new2>'},
        '1_4': {'image_path': '../hw2_data/textual_inversion/1/2019-11-08_cover_book-project.jpg', 'prompt': 'A photo of <new2>'}}
        """
        self.data = list(training_data.values())

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        image = Image.open(item["image_path"]).convert("RGB")
        prompt = item["prompt"]

        if self.transform:
            image = self.transform(image)
        #tokens_ID = model.cond_stage_model.tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
        tokens_ID = model.cond_stage_model.tokenizer(prompt, padding="max_length", max_length=model.cond_stage_model.tokenizer.model_max_length, truncation=True, return_tensors="pt")
        return image, tokens_ID

## Read input.json

In [11]:
# ============================================ READ INPUT.JSON ============================================
# Read input.json
f = open(jsonfile)
inputjson = json.load(f)
# get token name
if train_concept == "dog":
  token_names = [inputjson["0"]["token_name"]] # token_names of concept 0
else:
  token_names = [inputjson["1"]["token_name"]] # token_names of concept 1
#token_names = [inputjson["0"]["token_name"]] # token_names of concept 0
# image_dirs
if train_concept == "dog":
  image_dirs = [dog_dir] # token_names of concept 0
else:
  image_dirs = [david_revoy_dir] # token_names of concept 1

## LOAD Model

In [12]:
# ============================================ LOAD MODEL ============================================
# Load the model(ref: txt2img.py) & the pretrain weight using config
config = OmegaConf.load("configs/stable-diffusion/v1-inference.yaml")
model = load_model_from_config(config, f"{ckpt}")
model = model.to(device)
sampler = DPMSolverSampler(model)

Loading model from ldm/models/stable-diffusion-v1/model.ckpt
Global Step: 470000
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


Downloading: "https://github.com/DagnyT/hardnet/raw/master/pretrained/train_liberty_with_aug/checkpoint_liberty_with_aug.pth" to /root/.cache/torch/hub/checkpoints/checkpoint_liberty_with_aug.pth


  0%|          | 0.00/5.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

add special tokens and intial them using existing tokens

In [13]:
# Add pseudo-tokens special tokens: <new1> and <new2> and modify token_embeddings (49408, 768) -> (49410, 768)
model.cond_stage_model.tokenizer.add_tokens(token_names)
model.cond_stage_model.transformer.resize_token_embeddings(len(model.cond_stage_model.tokenizer))

if fine_tune:
  # Load the saved embeddings
  loaded_embeddings_david_revoy = torch.load("scripts/trained_embedding/fine_tune/trained_embeddings_david_revoy_epoch130.pt", map_location=device)
  model.cond_stage_model.transformer.get_input_embeddings().weight[-1] = loaded_embeddings_david_revoy
else:
  # Initialize special token embedding with copy from another existing token (like "photo")
  token_id_new = model.cond_stage_model.tokenizer.convert_tokens_to_ids(token_names)[0]
  existing_token_id = model.cond_stage_model.tokenizer.convert_tokens_to_ids(["cartoon"])[0]  # Use "photo" as a reference
  # Copy the "animal" token embedding for special token: initial token embedding can randomly choose
  model.cond_stage_model.transformer.get_input_embeddings().weight.data[token_id_new] = model.cond_stage_model.transformer.get_input_embeddings().weight.data[existing_token_id]
  print(existing_token_id)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
print(token_names)

['<new2>']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## LOAD TRAINING DATA

In [15]:
# ============================================ LOAD TRAINING DATA ============================================
# Define preprocessing for loaded images
preprocess = transforms.Compose([
    transforms.Resize((H, W)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=10),
    transforms.ToTensor(),
])
train_dataset = TextualInversionDataset(token_names, image_dirs, preprocess)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

image, prompt = next(iter(train_dataloader))  #get first batch's image & mask
print("image.shape: ", image.shape) # torch.Size([BS, 3, 512, 512])
print(prompt['input_ids'].shape) # torch.Size([BS, 1, 77])
print(len(train_dataloader))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


image.shape:  torch.Size([1, 3, 512, 512])
torch.Size([1, 1, 77])
5


In [16]:
# ============================================ TRAINING SETUP ============================================
for param in model.parameters():
    param.requires_grad = True
# Initialize optimizer and scheduler
# only train special token embedding
optimizer = optim.Adam(model.cond_stage_model.transformer.get_input_embeddings().parameters(), lr=0.001, weight_decay=1e-6)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=500, gamma=0.5, last_epoch=-1) # update lr each batch
embeddings = model.cond_stage_model.transformer.get_input_embeddings().weight
print(embeddings.requires_grad)

True


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
# ============================================ TRAINING LOOP ============================================
min_loss = float('inf')
for epoch in range(n_epoch):
    model.train()
    total_loss = 0
    for batch_idx, (image, token_ID) in enumerate(train_dataloader):
      optimizer.zero_grad()
      # get the latent from first stage encoder
      image = image.to(device) # torch.Size([BS, 3, 512, 512])
      latent_0 = encode_image_to_latent(image) #torch.Size([BS, 4, 64, 64])

      # get the output embedding from text encoder
      token_ID = token_ID['input_ids'].to(device) # torch.Size([BS, 1, 77])
      token_ID = token_ID.view(BATCH_SIZE, 77) # torch.Size([BS, 77])
      outputs = model.cond_stage_model.transformer(input_ids=token_ID)
      z = outputs[0] # torch.Size([BS, 77, 768])

      # t ~ Uniform({1,...,T})
      t_np = np.random.randint(0, model.num_timesteps, size=(BATCH_SIZE,)) # Sample t from Uniform({0, ..., T-1})
      t = torch.from_numpy(t_np)
      t = t.to(device)

      # epsilon ~ N(0, 1)
      noise = torch.randn_like(latent_0).to(device) # torch.Size([BS, 4, 64, 64])

      # 一步登天: get latent_t
      latent_t = model.q_sample(x_start=latent_0, t=t, noise=noise) # torch.Size([BS, 4, 64, 64])

      # Predict the noise
      noise_pred = model.apply_model(x_noisy=latent_t, t=t, cond=z) # torch.Size([BS, 4, 64, 64])


      # Compute loss (e.g., mean squared error between predicted embeddings and target embeddings)
      loss = model.get_loss(noise_pred, noise)

      # Backward pass and optimization step
      loss.backward()


      embeddings = model.cond_stage_model.transformer.get_input_embeddings().weight
      if embeddings.grad is not None:
        # Zero out gradients except for the last two rows using a for loop
        for i in range(embeddings.grad.shape[0] - 1):
          embeddings.grad[i].zero_()

      optimizer.step()
      # Track total loss
      total_loss += loss.item()



    total_loss /= len(train_dataloader)
    print(f'Epoch {epoch + 1}/{n_epoch} - Loss: {total_loss:.4f}')
    print(f'Learning rate {scheduler.get_last_lr()[0]}')

    # linear step LR
    scheduler.step()

    # Save the trained token embeddings
    if epoch % (n_epoch//50) == 0:
      trained_embedding = model.cond_stage_model.transformer.get_input_embeddings().weight[-1:].detach().cpu()  # Assuming <new1> and <new2> are the last tokens
      if fine_tune:
        torch.save(trained_embedding, f"scripts/trained_embedding/fine_tune_2/trained_embeddings_{train_concept}_epoch{epoch}.pt")
      else:
        torch.save(trained_embedding, f"scripts/trained_embedding/trained_embeddings_{train_concept}_epoch{epoch}.pt")
      print(f"Token embeddings saved at epoch {epoch}!")
    elif epoch == n_epoch - 1:
      trained_embedding = model.cond_stage_model.transformer.get_input_embeddings().weight[-1:].detach().cpu()  # Assuming <new1> and <new2> are the last tokens
      if fine_tune:
        torch.save(trained_embedding, f"scripts/trained_embedding/fine_tune_2/trained_embeddings_{train_concept}_epoch{epoch}.pt")
      else:
        torch.save(trained_embedding, f"scripts/trained_embedding/trained_embeddings_{train_concept}_epoch{epoch}.pt")
      print(f"Token embeddings saved at epoch {epoch}!")



print("Training complete!")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Epoch 1/500 - Loss: 0.1317
Learning rate 0.001
Token embeddings saved at epoch 0!
Epoch 2/500 - Loss: 0.3107
Learning rate 0.001
Epoch 3/500 - Loss: 0.1948
Learning rate 0.001
Epoch 4/500 - Loss: 0.1583
Learning rate 0.001
Epoch 5/500 - Loss: 0.3852
Learning rate 0.001
Epoch 6/500 - Loss: 0.1885
Learning rate 0.001
Epoch 7/500 - Loss: 0.1594
Learning rate 0.001
Epoch 8/500 - Loss: 0.1608
Learning rate 0.001
Epoch 9/500 - Loss: 0.1206
Learning rate 0.001
Epoch 10/500 - Loss: 0.1983
Learning rate 0.001
Epoch 11/500 - Loss: 0.1448
Learning rate 0.001
Token embeddings saved at epoch 10!
Epoch 12/500 - Loss: 0.1892
Learning rate 0.001
Epoch 13/500 - Loss: 0.0922
Learning rate 0.001
Epoch 14/500 - Loss: 0.1440
Learning rate 0.001
Epoch 15/500 - Loss: 0.3879
Learning rate 0.001
Epoch 16/500 - Loss: 0.0525
Learning rate 0.001
Epoch 17/500 - Loss: 0.1882
Learning rate 0.001
Epoch 18/500 - Loss: 0.0983
Learning rate 0.001
Epoch 19/500 - Loss: 0.0752
Learning rate 0.001
Epoch 20/500 - Loss: 0.279